<a href="https://colab.research.google.com/github/SIDIBEMoussa/NLP-Trainings/blob/main/03_Exercice_LM_classification_and_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [3]:
! git clone https://github.com/selva86/datasets.git

Cloning into 'datasets'...
remote: Enumerating objects: 483, done.
remote: Total 483 (delta 0), reused 0 (delta 0), pack-reused 483
Receiving objects: 100% (483/483), 43.65 MiB | 26.39 MiB/s, done.
Resolving deltas: 100% (156/156), done.


In [6]:
import pathlib, os

In [8]:
path = "/content/datasets/lda_sports_politics_docs"

file_directory = pathlib.Path(path).exists()

if not file_directory:

    !git clone https://github.com/selva86/datasets.git



text_files = os.listdir(path)

In [9]:
text_files

['shinzo abe.txt',
 'pizza.txt',
 'lee quan yew.txt',
 'queen elizabeth.txt',
 'cricket.txt',
 'idli.txt',
 'barack obama.txt',
 'baseball.txt',
 'badminton.txt',
 'noodles.txt',
 'tipu sultan.txt',
 'table tennis.txt',
 'pasta.txt',
 'narendra modi.txt',
 'dosa.txt']

In [12]:
articles=[]

for filename in text_files:

    article = pathlib.Path(path+'/'+filename).read_text()

    articles.append(article)

In [15]:
articles[0]

"Shinzō Abe (Abe Shinzō, IPA: born 21 September 1954) is the current Prime Minister of Japan, re-elected to the position in December 2012. Abe is also the President of the Liberal Democratic Party (LDP).\nAbe served for a year as Prime Minister, from 2006 to 2007. Hailing from a politically prominent family, at age 52, Abe became Japan's youngest post-war prime minister, and the first to be born after World War II, when he was elected by a special session of the National Diet in September 2006. Abe resigned on 12 September 2007, for health reasons. Abe was replaced by Yasuo Fukuda, beginning a string of five Prime Ministers, none of whom retained office for more than sixteen months, before Abe staged a political comeback.\nOn 26 September 2012, Abe defeated former Minister of Defense Shigeru Ishiba, in a run-off vote, to win the LDP Presidential Election. Following the LDP's landslide victory in the 2012 general election, Abe became the Prime Minister again. Abe is the first former Pri